In [1]:
import pandas as pd
import numpy as np
import os
import json
import spacy

2022-05-23 10:16:18.336560: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/mihaela/opt/openmpi/lib
2022-05-23 10:16:18.336597: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Data Loading


In [5]:
def read_json_file(path):
    f = open(path)
    json_content = json.load(f)
    f.close()
    return json_content

In [6]:
training_dataset_directory = 'datasets/rumoureval-2019-training-data/twitter-english'
test_dataset_directory = 'datasets/rumoureval-2019-test-data/twitter-en-test-data'

training_dataset_reddit_directory = 'datasets/rumoureval-2019-training-data/reddit-training-data'
test_dataset_reddit_directory = 'datasets/rumoureval-2019-test-data/reddit-test-data'

training_labels_json = 'datasets/rumoureval-2019-training-data/train-key.json'
training_labels_json_2 = 'datasets/rumoureval-2019-training-data/dev-key.json'
test_labels_json = 'datasets/final-eval-key.json'

training_labels_dict = read_json_file(training_labels_json)['subtaskaenglish']
training_labels_dict.update(read_json_file(training_labels_json_2)['subtaskaenglish'])
test_labels_dict = read_json_file(test_labels_json)['subtaskaenglish']

In [7]:
class Tweet:
    def __init__(self, post_content, post_id, parent_post_id=None, external_urls_count=0):
        self.post_content = post_content
        self.post_id = post_id
        self.category = None
        self.parent_post_id = parent_post_id
        self.external_urls = external_urls_count > 0
        self.user_metadata = None
        
    def add_category(self, category):
        self.category = category

In [8]:
class SourceTweet:
    def __init__(self, tweet: Tweet):
        self.tweet = tweet
        self.replies = []
        
    def add_reply(self, reply: Tweet):
        self.replies.append(reply)

In [9]:
def read_tweets_dataset(dataset_dir_path, labels_dict):
    topic_directories = [f.path for f in os.scandir(dataset_dir_path) if f.is_dir()]
    topic_to_tweets_map = {}  # {topic_name: [SourceTweet, ...]}

    for topic_dir in topic_directories:
        topic_name = topic_dir.split('/')[1]
        source_tweets = []
        
        tweets_paths = [f.path for f in os.scandir(topic_dir) if f.is_dir()]
        for tweet_dir in tweets_paths:
            source_tweet_path = [f.path for f in os.scandir(tweet_dir + '/source-tweet')][0]
            source_tweet_json = read_json_file(source_tweet_path)
            
            tweet = Tweet(source_tweet_json['text'], source_tweet_json['id'],
                              source_tweet_json['in_reply_to_status_id'],
                              len(source_tweet_json['entities']['urls']))
            
            source_tweet = SourceTweet(tweet)
            source_tweets.append(source_tweet)
            tweet.add_category("support")
            source_tweet.add_reply(tweet)

            #if not os.path.exists(topic_dir+'replies'):
            #    continue
            try:
                reply_tweets_paths = [f.path for f in os.scandir(tweet_dir + '/replies')]
            except:
                continue
            
            for reply_tweet_path in reply_tweets_paths:
                reply_tweet_json = read_json_file(reply_tweet_path)
                
                reply_tweet = Tweet(reply_tweet_json['text'], reply_tweet_json['id'],
                                        source_tweet.tweet.post_id, len(reply_tweet_json['entities']['urls']))
                reply_tweet.add_category(labels_dict[str(reply_tweet_json['id'])])
                source_tweet.add_reply(reply_tweet)
        
        topic_to_tweets_map[topic_name] = source_tweets
        
    return topic_to_tweets_map
       

In [10]:
def read_reddit_dataset(dataset_dir_path, labels_dict):
    topic_directories = [f.path for f in os.scandir(dataset_dir_path) if f.is_dir()]
    topic_to_tweets_map = {}  # {topic_name: [SourceTweet, ...]}

    for topic_dir in topic_directories:
        topic_name = topic_dir.split('/')[1]
        source_tweets = []
        
        source_tweet_path = [f.path for f in os.scandir(topic_dir + '/source-tweet')][0]
        source_tweet_json = read_json_file(source_tweet_path)

        content = source_tweet_json['data']['children'][0]['data']['title'] + ' ' + source_tweet_json['data']['children'][0]['data']['selftext']
        tweet = Tweet(content, source_tweet_json['data']['children'][0]['data']['id'], None, content.count("http"))

        source_tweet = SourceTweet(tweet)
        source_tweets.append(source_tweet)
        tweet.add_category("support")
        source_tweet.add_reply(tweet)

        #if not os.path.exists(topic_dir+'replies'):
        #    continue

        try:
            reply_tweets_paths = [f.path for f in os.scandir(topic_dir + '/replies')]
        except:
            continue
        
        for reply_tweet_path in reply_tweets_paths:
            reply_tweet_json = read_json_file(reply_tweet_path)
            
            if 'body' in reply_tweet_json['data']:
                reply_tweet = Tweet(reply_tweet_json['data']['body'], reply_tweet_json['data']['id'],
                                        source_tweet.tweet.post_id, reply_tweet_json['data']['body'].count('http'))
                reply_tweet.add_category(labels_dict[str(reply_tweet.post_id)])
                source_tweet.add_reply(reply_tweet)
                
        topic_to_tweets_map[topic_name] = source_tweets
        
    return topic_to_tweets_map

In [11]:
# Twitter
training_topic_to_tweets_map = read_tweets_dataset(training_dataset_directory, training_labels_dict)
test_topic_to_tweets_map = read_tweets_dataset(test_dataset_directory, test_labels_dict)

# Reddit
training_topic_to_reddit_map = read_reddit_dataset(training_dataset_reddit_directory, training_labels_dict)
test_topic_to_reddit_map = read_reddit_dataset(test_dataset_reddit_directory, test_labels_dict)

# Preprocessing

In [12]:
nlp = spacy.load("en_core_web_sm")

In [13]:
def preprocessing(sentence):
    lemmatizer = nlp.get_pipe("lemmatizer")        
    doc = nlp(sentence)
    lemmas = []
    for token in doc:
        if token.is_stop:
            continue
        elif token.pos_ == "NUM":
            lemmas.append('#')
        elif token.pos_ == "SYM":
            continue
        else:
            lemmas.append(token.lemma_.lower())
    return lemmas

In [14]:
def create_df(topic_map):
    rows = []
    for topic, source_tweets in topic_map.items():
        for source_tweet in source_tweets:
            tokenized_source_tweet = preprocessing(source_tweet.tweet.post_content)
            for reply in source_tweet.replies:
                tokenized_reply = preprocessing(reply.post_content)
                rows.append((topic, source_tweet.tweet.post_content, reply.post_content, tokenized_source_tweet, tokenized_reply, reply.external_urls, reply.category))
    return pd.DataFrame(rows, columns=['topic', 'original_source_tweet', 'original_reply', 'source_tweet', 'reply', 'external_urls', 'category'])

## DataFrames

In [15]:
training_tweets_df = create_df(training_topic_to_tweets_map)
test_tweets_df = create_df(test_topic_to_tweets_map)
training_reddit_df = create_df(training_topic_to_reddit_map)
test_reddit_df = create_df(test_topic_to_reddit_map)

In [16]:
training_tweets_df

,topic,original_source_tweet,original_reply,source_tweet,reply,external_urls,category
0,rumoureval-2019-training-data,"In response to inquiries, we can confirm that ...","In response to inquiries, we can confirm that ...","[response, inquiry, ,, confirm, prince, perfor...","[response, inquiry, ,, confirm, prince, perfor...",False,support
1,rumoureval-2019-training-data,"In response to inquiries, we can confirm that ...",@brofromanother @LiveNationON @masseyhall WHAT...,"[response, inquiry, ,, confirm, prince, perfor...","[@brofromanother, @livenationon, @masseyhall, ...",False,comment
2,rumoureval-2019-training-data,"In response to inquiries, we can confirm that ...",@LiveNationON Soo... Its tomorrow night!? #Pri...,"[response, inquiry, ,, confirm, prince, perfor...","[@livenationon, soo, ..., tomorrow, night, !, ...",False,query
3,rumoureval-2019-training-data,"In response to inquiries, we can confirm that ...",There ya go. @KiSS925 RT @LiveNationON: In res...,"[response, inquiry, ,, confirm, prince, perfor...","[ya, ., @kiss925, rt, @livenationon, :, respon...",False,comment
4,rumoureval-2019-training-data,"In response to inquiries, we can confirm that ...",@LiveNationON @masseyhall it's probably @3RDEY...,"[response, inquiry, ,, confirm, prince, perfor...","[@livenationon, @masseyhall, probably, @3rdeye...",False,comment
...,...,...,...,...,...,...,...
98,rumoureval-2019-training-data,Prince is playing a secret show in Toronto ton...,Prince is playing a secret show in Toronto ton...,"[prince, play, secret, toronto, tonight, ., ,...","[prince, play, secret, toronto, tonight, ., ,...",False,support
99,rumoureval-2019-training-data,Prince is playing a secret show in Toronto ton...,"@NobleRobel Girl, don't be mean. Age ain't n...","[prince, play, secret, toronto, tonight, ., ,...","[@noblerobel, girl, ,, mean, ., , age, be, n...",False,comment
100,rumoureval-2019-training-data,Prince is playing a secret show in Toronto ton...,@mikeyerxa you should send Prince a set of ten...,"[prince, play, secret, toronto, tonight, ., ,...","[@mikeyerxa, send, prince, set, tennis, ball, ...",False,comment
101,rumoureval-2019-training-data,Prince is playing a secret show in Toronto ton...,"@mikeyerxa EXCUSE ME, WHERE'S MY INVITE (to th...","[prince, play, secret, toronto, tonight, ., ,...","[@mikeyerxa, excuse, ,, invite, (, ,, duh, )]",False,comment


In [17]:
test_tweets_df

,topic,original_source_tweet,original_reply,source_tweet,reply,external_urls,category
0,rumoureval-2019-test-data,BREAKING: Illegal Muslim From Iran Arrested Fo...,BREAKING: Illegal Muslim From Iran Arrested Fo...,"[breaking, :, illegal, muslim, iran, arrest, s...","[breaking, :, illegal, muslim, iran, arrest, s...",True,support
1,rumoureval-2019-test-data,BREAKING: Illegal Muslim From Iran Arrested Fo...,@jjauthor Article is dated in October? It's a...,"[breaking, :, illegal, muslim, iran, arrest, s...","[@jjauthor, article, date, october, ?, , para...",True,deny
2,rumoureval-2019-test-data,BREAKING: Illegal Muslim From Iran Arrested Fo...,"@jjauthor @GrizzleMeister Great Example, CaGov...","[breaking, :, illegal, muslim, iran, arrest, s...","[@jjauthor, @grizzlemeister, great, example, ,...",True,comment
3,rumoureval-2019-test-data,BREAKING: Illegal Muslim From Iran Arrested Fo...,@jjauthor I’m sure the sanctuary state will fo...,"[breaking, :, illegal, muslim, iran, arrest, s...","[@jjauthor, sure, sanctuary, state, forgive, .]",False,comment
4,rumoureval-2019-test-data,BREAKING: Illegal Muslim From Iran Arrested Fo...,@jjauthor FAKE...you gotta do better Janie.,"[breaking, :, illegal, muslim, iran, arrest, s...","[@jjauthor, fake, ..., get, to, well, janie, .]",False,comment
5,rumoureval-2019-test-data,BREAKING: Illegal Muslim From Iran Arrested Fo...,@jjauthor @GrizzleMeister Why am I not surpris...,"[breaking, :, illegal, muslim, iran, arrest, s...","[@jjauthor, @grizzlemeister, surprised, ,, cou...",False,support
6,rumoureval-2019-test-data,BREAKING: Illegal Muslim From Iran Arrested Fo...,@jjauthor @GrizzleMeister Couldn't get past th...,"[breaking, :, illegal, muslim, iran, arrest, s...","[@jjauthor, @grizzlemeister, past, ad, read]",False,comment
7,rumoureval-2019-test-data,BREAKING: Illegal Muslim From Iran Arrested Fo...,@jjauthor This makes one wonder about the orig...,"[breaking, :, illegal, muslim, iran, arrest, s...","[@jjauthor, make, wonder, origin, fire, past, ...",False,comment
8,rumoureval-2019-test-data,BREAKING: Illegal Muslim From Iran Arrested Fo...,BREAKING: Illegal Muslim From Iran Arrested Fo...,"[breaking, :, illegal, muslim, iran, arrest, s...","[breaking, :, illegal, muslim, iran, arrest, s...",True,support
9,rumoureval-2019-test-data,BREAKING: Illegal Muslim From Iran Arrested Fo...,@WhoWolfe shut down #MSA &amp; visa's,"[breaking, :, illegal, muslim, iran, arrest, s...","[@whowolfe, shut, msa, &, amp, ;, visa]",False,comment


In [18]:
training_reddit_df

,topic,original_source_tweet,original_reply,source_tweet,reply,external_urls,category
0,rumoureval-2019-training-data,Debunk This: Microwaves are bad because microw...,Debunk This: Microwaves are bad because microw...,"[debunk, :, microwave, bad, microwaved, water,...","[debunk, :, microwave, bad, microwaved, water,...",False,support
1,rumoureval-2019-training-data,Debunk This: Microwaves are bad because microw...,&gt;It has been known for some years that the ...,"[debunk, :, microwave, bad, microwaved, water,...","[&, gt;it, know, year, problem, microwaved, ra...",False,deny
2,rumoureval-2019-training-data,Debunk This: Microwaves are bad because microw...,"Where was her control ?, why did she prune new...","[debunk, :, microwave, bad, microwaved, water,...","[control, ?, ,, prune, newly, grow, plant, ?, ...",False,query
3,rumoureval-2019-training-data,Debunk This: Microwaves are bad because microw...,Also different antigens in blood become more o...,"[debunk, :, microwave, bad, microwaved, water,...","[different, antigen, blood, active, different,...",False,comment
4,rumoureval-2019-training-data,Debunk This: Microwaves are bad because microw...,[This has been debunked by Snopes](http://www....,"[debunk, :, microwave, bad, microwaved, water,...","[[, debunk, snopes](http://www.snopes.com, sci...",True,deny
5,rumoureval-2019-training-data,Debunk This: Microwaves are bad because microw...,Mythbusters tackled the plant thing and found ...,"[debunk, :, microwave, bad, microwaved, water,...","[mythbuster, tackle, plant, thing, find, bust,...",False,deny
6,rumoureval-2019-training-data,Debunk This: Microwaves are bad because microw...,"1. Microwaved water is just normal water, but ...","[debunk, :, microwave, bad, microwaved, water,...","[1, ., microwaved, water, normal, water, ,, ho...",False,deny
7,rumoureval-2019-training-data,Debunk This: Microwaves are bad because microw...,&gt;The body doesn't give a tinkers cuss about...,"[debunk, :, microwave, bad, microwaved, water,...","[&, gt;the, body, tinker, cuss, dna, food, ing...",False,comment
8,rumoureval-2019-training-data,Debunk This: Microwaves are bad because microw...,Apparently her control was another plant which...,"[debunk, :, microwave, bad, microwaved, water,...","[apparently, control, plant, water, water, boi...",False,comment


In [19]:
test_reddit_df

,topic,original_source_tweet,original_reply,source_tweet,reply,external_urls,category
0,rumoureval-2019-test-data,Liverpool students campaign to remove Gladston...,Liverpool students campaign to remove Gladston...,"[liverpool, student, campaign, remove, gladsto...","[liverpool, student, campaign, remove, gladsto...",False,support
1,rumoureval-2019-test-data,Liverpool students campaign to remove Gladston...,A girl I dated in college for a bit was electe...,"[liverpool, student, campaign, remove, gladsto...","[girl, date, college, bit, elect, su, presiden...",False,comment
2,rumoureval-2019-test-data,Liverpool students campaign to remove Gladston...,I think we had ~15% for an NUS disaffiliation ...,"[liverpool, student, campaign, remove, gladsto...","[think, ~15, %, nus, disaffiliation, referendu...",False,comment
3,rumoureval-2019-test-data,Liverpool students campaign to remove Gladston...,Well while I am sure this is a vocal minority ...,"[liverpool, student, campaign, remove, gladsto...","[sure, vocal, minority, need, stop, judge, peo...",False,comment
4,rumoureval-2019-test-data,Liverpool students campaign to remove Gladston...,"I hate Gladstone, and even I think this is non...","[liverpool, student, campaign, remove, gladsto...","[hate, gladstone, ,, think, nonsense, high, or...",False,support
...,...,...,...,...,...,...,...
192,rumoureval-2019-test-data,Liverpool students campaign to remove Gladston...,If you are campaigning for this then you're no...,"[liverpool, student, campaign, remove, gladsto...","[campaign, liberal, ., try, -, write, history,...",False,comment
193,rumoureval-2019-test-data,Liverpool students campaign to remove Gladston...,"You can and probably should, but student polit...","[liverpool, student, campaign, remove, gladsto...","[probably, ,, student, politician, real, polit...",False,comment
194,rumoureval-2019-test-data,Liverpool students campaign to remove Gladston...,Mostly on the dodgy side of the web (download ...,"[liverpool, student, campaign, remove, gladsto...","[dodgy, web, (, download, site, link, ), ,, we...",False,comment
195,rumoureval-2019-test-data,Liverpool students campaign to remove Gladston...,So?,"[liverpool, student, campaign, remove, gladsto...",[?],False,comment


## LSTM network for stance classification

In [30]:
from tensorflow import keras
from keras import layers
from keras.layers import Input, LSTM, Embedding, Dense, Bidirectional
import keras.preprocessing.sequence as seq
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder


In [21]:
training_tweets_df['original_source_tweet']

0      In response to inquiries, we can confirm that ...
1      In response to inquiries, we can confirm that ...
2      In response to inquiries, we can confirm that ...
3      In response to inquiries, we can confirm that ...
4      In response to inquiries, we can confirm that ...
                             ...                        
98     Prince is playing a secret show in Toronto ton...
99     Prince is playing a secret show in Toronto ton...
100    Prince is playing a secret show in Toronto ton...
101    Prince is playing a secret show in Toronto ton...
102    Prince is playing a secret show in Toronto ton...
Name: original_source_tweet, Length: 103, dtype: object

In [22]:
training_tweets_df['original_reply']

0      In response to inquiries, we can confirm that ...
1      @brofromanother @LiveNationON @masseyhall WHAT...
2      @LiveNationON Soo... Its tomorrow night!? #Pri...
3      There ya go. @KiSS925 RT @LiveNationON: In res...
4      @LiveNationON @masseyhall it's probably @3RDEY...
                             ...                        
98     Prince is playing a secret show in Toronto ton...
99     @NobleRobel Girl, don't be mean.   Age ain't n...
100    @mikeyerxa you should send Prince a set of ten...
101    @mikeyerxa EXCUSE ME, WHERE'S MY INVITE (to th...
102    @halihamilton @mikeyerxa gonna try to make it ...
Name: original_reply, Length: 103, dtype: object

In [32]:
# concatenating tweet input and tweet replies
training_tweets_df['source_reply'] = training_tweets_df['original_source_tweet'] + training_tweets_df['original_reply']
test_tweets_df['source_reply'] = test_tweets_df['original_source_tweet'] + test_tweets_df['original_reply']

print(training_tweets_df['source_reply'])
print(test_tweets_df['source_reply'])

0      In response to inquiries, we can confirm that ...
1      In response to inquiries, we can confirm that ...
2      In response to inquiries, we can confirm that ...
3      In response to inquiries, we can confirm that ...
4      In response to inquiries, we can confirm that ...
                             ...                        
98     Prince is playing a secret show in Toronto ton...
99     Prince is playing a secret show in Toronto ton...
100    Prince is playing a secret show in Toronto ton...
101    Prince is playing a secret show in Toronto ton...
102    Prince is playing a secret show in Toronto ton...
Name: source_reply, Length: 103, dtype: object
0     BREAKING: Illegal Muslim From Iran Arrested Fo...
1     BREAKING: Illegal Muslim From Iran Arrested Fo...
2     BREAKING: Illegal Muslim From Iran Arrested Fo...
3     BREAKING: Illegal Muslim From Iran Arrested Fo...
4     BREAKING: Illegal Muslim From Iran Arrested Fo...
5     BREAKING: Illegal Muslim From Iran Arres

In [38]:

vocabulary_size = 300000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(training_tweets_df['source_reply'])
train_sequences = tokenizer.texts_to_sequences(training_tweets_df['source_reply'])

tokenizer.fit_on_texts(training_tweets_df['source_reply'])
test_sequences = tokenizer.texts_to_sequences(training_tweets_df['source_reply'])


X_train = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=80, dtype='int32')

encoder_train = LabelEncoder()
encoder_train.fit(training_tweets_df['category'])
y_train = np_utils.to_categorical(encoder_train.transform(training_tweets_df['category']))


print(X_train)
print(y_train)


[[  0   0   0 ...  14  15   8]
 [  0   0   0 ... 209 210  53]
 [  0   0   0 ... 116   1  15]
 ...
 [  0   0   0 ... 487 173 488]
 [  0   0   0 ...  38 492 493]
 [  0   0   0 ...  37   7 497]]
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.

In [41]:
model = keras.Sequential()

model.add(Embedding(20000, 100, input_length=80))
model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(keras.layers.Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print(model.summary())

model.fit(X_train, y_train, validation_split=0.4, epochs=20)



Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 80, 100)           2000000   
                                                                 
 bidirectional_6 (Bidirectio  (None, 200)              160800    
 nal)                                                            
                                                                 
 dense_11 (Dense)            (None, 4)                 804       
                                                                 
Total params: 2,161,604
Trainable params: 2,161,604
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
2/2 [==============================] - 6s 855ms/step - loss: 1.3841 - categorical_accuracy: 0.2951 - val_loss: 1.3366 - val_categorical_accuracy: 0.7143
Epoch 2/20
2/2 [==============================] - 0s 288ms/step - loss: 